In [1]:
import os

# os.environ['JAVA_HOME'] = '/etc/alternatives/java_sdk'
os.environ["PYSPARK_PYTHON"] = os.environ['HOME'] + "/anaconda3/bin/python"
os.environ['SPARK_HOME'] = os.environ['HOME'] + '/spark-2.4.4-bin-hadoop2.7'  # Download spark from https://spark.apache.org/downloads.html
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"

In [2]:
# os.environ['PYTHONPATH'] = ';'.join([
#     os.environ['SPARK_HOME'] + '/python',
#     os.environ["PYLIB"] + '/py4j-0.10.7-src.zip',
#     os.environ.get('PYTHONPATH', '')
# ])

In [3]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [4]:
spark = (
    SparkSession
    .builder
    .appName("Python Spark SQL basic example")
    .master('local[64]')
    .config("spark.driver.memory", "200G")
    .config("spark.executor.memory", "16G")
    .config('spark.driver.maxResultSize', '200G')
    .getOrCreate()
)

In [5]:
%%time
df = spark.read.json('../data/ipython_notebooks_python_and_R_scripts_imports_full/*.json')

CPU times: user 4.54 ms, sys: 3.38 ms, total: 7.92 ms
Wall time: 15.6 s


In [6]:
from pyspark.sql.functions import countDistinct

In [7]:
%%time
counts = df.groupBy('script_type').agg(countDistinct('repo_name'))

CPU times: user 1.73 ms, sys: 1.59 ms, total: 3.32 ms
Wall time: 51.9 ms


In [8]:
%%time
c = counts.collect()

CPU times: user 6.55 ms, sys: 6.55 ms, total: 13.1 ms
Wall time: 23.6 s


In [9]:
cc = counts.toPandas()

In [10]:
cc

,script_type,count(DISTINCT repo_name)
0,R,20756
1,Python,562209
2,IPython,23776


In [7]:
# %%time
# # 34.5G 4:00pm
# pdf = df.toPandas()

CPU times: user 22min 19s, sys: 5min, total: 27min 20s
Wall time: 42min 14s


In [14]:
import gc
gc.collect()

951

<h1><span style="color:red;">Store full data to parquet file if we want to load the full data to pandas</span></h1>

```python
%%time
df.write.parquet('ipython_notebooks_python_and_R_scripts_imports_full.parquet')

# CPU times: user 13.9 ms, sys: 20.1 ms, total: 34 ms
# Wall time: 46.2 s

# Directly converting the full dataframe to pandas is really slow!
# %%time
# # 34.5G 4:00pm
# pdf = df.toPandas()
# # CPU times: user 22min 19s, sys: 5min, total: 27min 20s
# # Wall time: 42min 14s

```